# ADVENT OF CODE 2023 - DAY 7

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
with open(r"/kaggle/input/advent-data-7/Advent07.txt") as file:
    contents = file.read().splitlines()
    
data = contents

data = ["32T3K 765",
"T55J5 684",
"KK677 28",
"KTJJT 220",
"QQQJA 483",
]

Handtypes:

*     Five of a kind, where all five cards have the same label: AAAAA
*     Four of a kind, where four cards have the same label and one card has a different label: AA8AA
*     Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
*     Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
*     Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
*     One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
*     High card, where all cards' labels are distinct: 23456

Hands are primarily ordered based on type; for example, every full house is stronger than any three of a kind.
If two hands have the same type, a second ordering rule takes effect. Start by comparing the first card in each hand.
A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2.

In [3]:
import collections as C

result = 0

value = {}
for i,num in enumerate(["2","3","4","5","6","7","8","9","T","J","Q","K","A"]):
    value[num] = i+1
print(value, "\n")

cards = {}

for x in range(len(data)):
    hand = data[x].split()[0]
    bid = data[x].split()[1]
    n = C.Counter(hand).most_common()
    hand_value = 0
    if n[0][1] == 1:
        for sym in hand:
            if value[sym] > hand_value:
                hand_value = value[sym]
    else:
        if n[0][1] == 2 and n[1][1] != 2:
            hand_value = 15
        if n[0][1] == 2 and n[1][1] == 2:
            hand_value = 20
        if n[0][1] == 3 and n[1][1] != 2:
            hand_value = 30
        if n[0][1] == 3 and n[1][1] == 2:
            hand_value = 40
        if n[0][1] == 4:
            hand_value = 50
        if n[0][1] == 5:
            hand_value = 60
        
    cards[hand] = bid, hand_value
    # print(f"Hand: {hand}, Bid: {bid}, Amounts: {n}, Handvalue: {hand_value}")
    
sorted_cards = pd.DataFrame.from_dict(dict(sorted(cards.items(), key=lambda x:x[1][1])), orient="index")
sorted_cards = sorted_cards.reset_index()
sorted_cards.rename(columns={"index":"Hand",0:"Bid", 1:"Value"}, inplace=True)

{'2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, 'T': 9, 'J': 10, 'Q': 11, 'K': 12, 'A': 13} 



In [4]:
def sorter(item):
    first = value[item[0]]
    second = value[item[1]]
    third = value[item[2]]
    fourth = value[item[3]]
    fifth = value[item[4]]  
    return(first,second,third,fourth,fifth)

list_cards = []

for v in sorted_cards["Value"].unique():
    val = []
    for case in sorted_cards[sorted_cards["Value"]==v].values:
        # print(case)
        val.append(case[0])
    val.sort(key=sorter)
    list_cards.append(val)

v_dict = dict(sorted(cards.items(), key=lambda x:x[1][1]))
bid = []
for x in sum(list_cards,[]):
    a = v_dict[x][0]
    bid.append(a)

sorted_cards["Hand"] = sum(list_cards,[])
sorted_cards["Bid"] = bid
sorted_cards["Rank"] = sorted_cards["Value"].rank(method="first")
sorted_cards["Winnings"] = sorted_cards["Bid"].astype(int)*sorted_cards["Rank"].astype(int)
print(sorted_cards["Winnings"].sum())

250736116


In [5]:
print(sorted_cards)

      Hand  Bid  Value    Rank  Winnings
0    43682  205      7     1.0       205
1    42589  818      8     2.0      1636
2    45697  346      8     3.0      1038
3    63259  518      8     4.0      2072
4    72569  737      8     5.0      3685
..     ...  ...    ...     ...       ...
995  AA9AA  337     50   996.0    335652
996  AAA7A  571     50   997.0    569287
997  AAAJA  550     50   998.0    548900
998  AAAA6  986     50   999.0    985014
999  JJJJJ  517     60  1000.0    517000

[1000 rows x 5 columns]


# Solution from the subreddit because i couldnt figure out where i was mistake was...

In [6]:
data_raw = data

print(
    sum(
        (rank0 + 1) * bid
        for rank0, (*_, bid) in enumerate(
            sorted(
                (
                    max(map(hand.count, hand)),
                    -len(set(hand)),
                    *map("23456789TJQKA".index, hand),
                    int(str_bid),
                )
                for hand, str_bid in map(str.split, data_raw)
            )
        )
    )
)

250946742
